# Build a Recommendation System for Purchase Data

The scope of this notebook is 

- Prepare the Scoring Function
- Unit Test the Score
- Build Flask Scoring App and deploy as a Web End point
- Build Dash App and deploy as Interactive Web Services

The business case is an in-store app allowing its customers to place orders before they even have to walk into the store.
When a customer provides its own id and the app suggests

- Personalized recommendation with ranked list of items (product IDs) that the user is most likely to want to put in his/her (empty) “basket”

Assuming that the scenario is ModelOps 0. 

Then: 

1. Data scientists hand over a trained model as an artifact to the engineering team for deployement
2. The handoff can include putting the trained model in the models registry
3. The Scoring process is in Batch on a sigle EC2 instance

We have to reproduce the required development enviroment

0. We get the last version of Champion Model (optional)

1. Define Scoring Functions: Batch scoring is the main assumption

    - Define the get_top_items function 
    - Define the get_top_n_ui function
    
2. Unit Test 

3. Deploy Model as Scoring Web EndPoint

4. Deploy Model as Interactive Web Service


## Settings

### Import libraries

In [47]:
#Data
import sqlalchemy as sql

#Data Science
import numpy as np
import pandas as pd
import sklearn
import scipy
import surprise
from surprise import dump

#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient

#ML engineering
import flask
from flask import Flask, jsonify, Response
import gunicorn

#Utils
import os
import glob
import shutil
import logging
from collections import defaultdict
import configparser
import json
import pickle
import unittest
import docker
import pprint
import time
import requests

#Settings
from pprint import pprint
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

### Set enviroment variables

In [2]:
# Enviroment variables
outmodels = '../models/'
app_folder = '../src/score_endpoint'
dash_folder = '../score_interactive_endpoint'
docker_folder = './docker/'

# Set dbconnection variables
dbconnPath = './dbconn.properties'
config = configparser.RawConfigParser()
config.read(dbconnPath)
params = config
db_host=params.get('CONN', 'host')
db_port=params.get('CONN', 'port')
db_user=params.get('CONN', 'user')
db_pwd=params.get('CONN', 'password')
db_name=params.get('CONN', 'database')

# Set connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'

### Download Model Artefact from Mlflow server

In [3]:
client = MlflowClient()
for regmodel in client.list_registered_models():
    regmodel_info = dict(regmodel)

# pprint(regmodel_info, indent=3)

champion=client.get_registered_model('Champion')
championid=champion.latest_versions[-1].run_id

art_list = [arts.path for arts in client.list_artifacts(championid, path=None)]

for art_path in art_list: 
    client.download_artifacts(championid, art_path, outmodels)

### Analyze the Model Artifact

In [4]:
modelpkl = [modelpath for modelpath in glob.glob(outmodels + 'model/*.pkl')][0]
modelpkl

predictions, algo = dump.load(modelpkl)

print('\n')
print('Sample of Predictions: ')
print('\n', predictions[0:10])
print('\n', 'Number of predictions:', len(predictions))



Sample of Predictions: 

 [Prediction(uid='100', iid='0', r_ui=1.0, est=1.4104866760238497, details={'was_impossible': False}), Prediction(uid='100', iid='118', r_ui=2.0, est=1.6290418315812665, details={'was_impossible': False}), Prediction(uid='100', iid='201', r_ui=1.0, est=1.0281647548199493, details={'was_impossible': False}), Prediction(uid='100', iid='24', r_ui=2.0, est=1.065194864590783, details={'was_impossible': False}), Prediction(uid='100', iid='27', r_ui=4.0, est=1.4686861264367166, details={'was_impossible': False}), Prediction(uid='100', iid='282', r_ui=6.0, est=1.2631693065495946, details={'was_impossible': False}), Prediction(uid='100', iid='51', r_ui=0.0, est=1.3060476975670121, details={'was_impossible': False}), Prediction(uid='100', iid='6', r_ui=0.0, est=1.8579018429484466, details={'was_impossible': False}), Prediction(uid='100', iid='62', r_ui=2.0, est=1.0656423124377505, details={'was_impossible': False}), Prediction(uid='100', iid='67', r_ui=3.0, est=1.55989

## Machine Learning Engineering

### Scoring Function

We have to return Top 10 Recommended Items by userid

#### Define the scoring function

In [5]:
def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: # user was not part of the trainset
        return 0

#### Unit test

In [6]:
# class TestScoreFunction(unittest.TestCase):
    
#     def setUp(self):
#         self.testcase = "100"
#         self.expected = ['6', '118', '67', '27', '0', '51', '282', '62', '24', '201']
    
#     def test_empty(self):
#         self.assertTrue(bool(get_top_n_ui(get_top(predictions), self.testcase)))

#     def test_basic(self):
#         self.assertEqual(get_top_n_ui(get_top(predictions), self.testcase), self.expected)
        
# unittest.main(argv = ['first-arg-is-ignored'], exit = False)

### Model Deployment: Scoring Web EndPoint and WSGI server 

Because we're testing, I need a app folder with the model and: 

1. app.py
2. wsgi.py
3. requirements.txt
3. Dockerfile

Then I will run the application with Docker Client and I will test it 

#### Create a app folder

In [7]:
if not os.path.exists(app_folder):
    os.makedirs(app_folder)

##### Copy the model

In [8]:
if not os.path.exists(app_folder + '/model'):
    shutil.copytree(src=outmodels + 'model', dst=app_folder + '/model')

In [9]:
os.chdir(app_folder)

##### Create app.py module

In [29]:
%%writefile app.py

# -*- coding: utf-8 -*-

'''
A basic Flask application for scoring web endpoint.
'''

import os
from collections import defaultdict
import pandas as pd
from surprise import dump

import flask
from flask import Flask, jsonify, Response
import warnings
warnings.filterwarnings("ignore")

#create an instance
app = Flask(__name__)

def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0

@app.route('/', methods=['GET'])
def health_checker():
    text = 'The Scoring application is ready!'
    return Response(text, status=200, mimetype='text/plain')

@app.route('/predict', methods=['GET','POST'])
def predictor():

    #Intiate variables
    data = defaultdict()
    data["success"] = False
    params = flask.request.args
    
    if 'uid' in params.keys():
        uid_toscore = str(params.get('uid'))
        model_path = locate_model(os.getcwd())
        predictions, _ = model_reader(model_path)
        uid_predictions = get_top_n_ui(get_top(predictions), uid_toscore)
        
        prediction_rank_lenght = len(uid_predictions)
        prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
        products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])

        data['response'] = products_recommended.to_dict()
        data['success'] = True
    
    return flask.jsonify(data)
            
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9999, debug=True)

Overwriting app.py


In [39]:
%%writefile app.py

# -*- coding: utf-8 -*-

'''
A basic Flask application for scoring web endpoint.
'''

import os
from collections import defaultdict
import pandas as pd
from surprise import dump

import flask
from flask import Flask, jsonify, Response
import warnings
warnings.filterwarnings("ignore")

#create an instance
app = Flask(__name__)

def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0

@app.route('/', methods=['GET'])
def health_checker():
    text = 'The Scoring application is ready!'
    return Response(text, status=200, mimetype='text/plain')

@app.route('/predict', methods=['GET','POST'])
def predictor():

    #Intiate variables
    data = defaultdict()
    data["success"] = False
    params = flask.request.args
    
    if 'uid' in params.keys():
        uid_toscore = str(params.get('uid'))
        model_path = locate_model(os.getcwd())
        predictions, _ = model_reader(model_path)
        uid_predictions = get_top_n_ui(get_top(predictions), uid_toscore)
        
        prediction_rank_lenght = len(uid_predictions)
        prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
        products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])

        data['response'] = products_recommended.to_dict()
        data['success'] = True
    
    return flask.jsonify(data)

Overwriting app.py


##### Create wsgi.py module

In [67]:
%%writefile wsgi.py

from app import app
import logging
import os

import warnings
warnings.filterwarnings("ignore")

if __name__ != '__main__':
    app.run(host='0.0.0.0', port=9999)
    #Return a logger named gunicorn.error
    gunicorn_logger = logging.getLogger('gunicorn.error')
    #Set app logging policies
    app.logger.handlers = gunicorn_logger.handlers
    app.logger.setLevel(gunicorn_logger.level)

def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

# setup model folder
model_folder = '/app/model'
if not os.path.isdir(model_folder):
    app.logger.info("ERROR: Can't find model folder in the file system")
    raise RuntimeError("Model Folder does not exist. Please check how you create the docker image")
    
# locate model pickle
model_path = locate_model(model_folder)
if model_path is None:
    app.logger.info("ERROR: Can't find model pickle file in the {}".format(model_path))
    raise RuntimeError("Can't find model pickle file in the {}!".format(model_path))

print('The Scoring Web Service is ready!')

Overwriting wsgi.py


In [68]:
%%writefile run_server.sh
#!/bin/sh

exec gunicorn --bind 0.0.0.0:9999 wsgi:app \
    --log-level=debug \
    --log-file=/var/log/gunicorn.log \
    --access-logfile=/var/log/gunicorn-access.log \
"$@"

Overwriting run_server.sh


##### Create the requirements.txt

In [69]:
print(flask.__version__)
print(gunicorn.__version__)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(surprise.__version__)

1.0.3
20.0.4
1.18.4
1.0.4
0.22.2.post1
1.1.0


In [70]:
%%writefile requirements.txt

numpy
pandas
scikit-surprise
flask
gunicorn

Overwriting requirements.txt


##### Create dockerfile for app enviroment

In [61]:
%%writefile Dockerfile

# Use an official Python runtime as parent image
FROM python:3
    
LABEL Scoring App = "Recommendation System - Python - Surprise"

# RUN apt-get update && apt-get install -y \
# python3-dev \
# build-essential    
        
# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Upgrade pip and install any needed packages specified in requirements.txt
RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# Make port 9999 available
EXPOSE 9999

#Entrypoint python exec
ENTRYPOINT [ "python" ]

#Run flask application
CMD [ "app.py" ]

Overwriting Dockerfile


In [62]:
%%writefile Dockerfile

# Use an official Python runtime as parent image
FROM python:3
    
LABEL Scoring App = "Recommendation System - Python - Surprise"

# RUN apt-get update && apt-get install -y \
# python3-dev \
# build-essential    
        
# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Upgrade pip and install any needed packages specified in requirements.txt
RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# Make port 9999 available
EXPOSE 9999

#Entrypoint python exec
ENTRYPOINT [ "/bin/sh" ]

#Run flask application
CMD [ "run_server.sh" ]

Overwriting Dockerfile


#### Run the Application with Python Docker Client

In [63]:
image_name = "score-flask_app:1"

#Client instance
dockercli = docker.DockerClient()

#Check for image
if not dockercli.images.list(image_name):
    #if not build it
    dockercli.images.build(path='.', tag = image_name)
else:
    dockercli.images.remove(image_name, force = True)
    dockercli.images.build(path='.', tag = image_name)
try:
    app_container = dockercli.containers.run(image_name, name='scoring_app_test', detach=True, ports={'9999/tcp': 9999})
    status = app_container.attrs
    print(status['State'])
    while (status['State']['Running'] == False):
        time.sleep(3)
        app_container.reload()
        status = app_container.attrs
        print(''.center(50, '-'))
        print(status['State'])
except RuntimeError as error:
    print(error)

{'Status': 'created', 'Running': False, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 0, 'ExitCode': 0, 'Error': '', 'StartedAt': '0001-01-01T00:00:00Z', 'FinishedAt': '0001-01-01T00:00:00Z'}
--------------------------------------------------
{'Status': 'exited', 'Running': False, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 0, 'ExitCode': 3, 'Error': '', 'StartedAt': '2020-06-06T10:14:04.455632311Z', 'FinishedAt': '2020-06-06T10:14:05.216807215Z'}
--------------------------------------------------
{'Status': 'exited', 'Running': False, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 0, 'ExitCode': 3, 'Error': '', 'StartedAt': '2020-06-06T10:14:04.455632311Z', 'FinishedAt': '2020-06-06T10:14:05.216807215Z'}
--------------------------------------------------
{'Status': 'exited', 'Running': False, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 0, 'ExitCode

KeyboardInterrupt: 

#### Test the Flask Scoring App as web endpoint

In [55]:
protocol = 'http'
server = '10.249.21.252'
port = '9999'

# Check that the container is available
check_request = requests.get(protocol + "://" + server + ":" + port + "/")
check_response = check_request.text

KeyboardInterrupt: 

In [35]:
print(check_response)

The Scoring application is ready!


In [36]:
params = {'uid': 100}

# Scoring new data
score_request = requests.get(protocol + "://" + server + ":" + port + "/predict", params=params)
score_response = score_request.text

In [37]:
print(score_response)

{
  "response": {
    "Product_Rank": {
      "0": " Product1", 
      "1": " Product2", 
      "2": " Product3", 
      "3": " Product4", 
      "4": " Product5", 
      "5": " Product6", 
      "6": " Product7", 
      "7": " Product8", 
      "8": " Product9"
    }, 
    "Product_id": {
      "0": "6", 
      "1": "118", 
      "2": "67", 
      "3": "27", 
      "4": "0", 
      "5": "51", 
      "6": "282", 
      "7": "62", 
      "8": "24"
    }
  }, 
  "success": true
}



#### Kill the container

In [66]:
# stop and remove the container
app_container.stop()
app_container.remove()

### Model Deployment: A basic Scoring Interactive Web Service

Because our scenario is going to have a on-site application, I've just prototype a basic Scoring Interactive Web Service.
Again I need an app folder with model and:

1. app.py
2. layout.py
3. callbacks.py
4. run.py

And again i'll run the app and I'll test it with Python Docker client


#### Create the app folder

In [10]:
if not os.path.exists(dash_folder):
    os.makedirs(dash_folder)

##### Copy the Model

In [11]:
if not os.path.exists(dash_folder + '/model'):
    shutil.copytree(src='../' + outmodels + 'model', dst=dash_folder + '/model')

In [12]:
os.chdir(dash_folder)

##### Create app.py module

In [180]:
%%writefile app.py
# -*- coding: utf-8 -*-

'''
The Dash instance module
'''

import os
import logging
from collections import defaultdict
import pandas as pd
from surprise import dump

import dash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions = True)
server = app.server

def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0

Overwriting app.py


##### Create layouts.py module

In [181]:
%%writefile layouts.py

'''
This module contains the layout of the application
'''

import dash_table
import dash_core_components as component
import dash_html_components as html

#Customerid Input
c_id_component = component.Input(id="uid", 
                                 type="text", 
                                 placeholder='100')

c_id_div = html.Div(id='customerid', children=[html.H6("Please enter your customer ID : "), 
                                               c_id_component,
                                               html.Br()], 
                   className="two columns")

#Prediction Output
predictions = dash_table.DataTable(id='table',
                                   css=[{'selector': 'table', 'rule': 'table-layout: fixed'}],
                                   style_as_list_view=True, 
                                   style_cell={'width': '30%', 'textAlign': 'left'},
                                   style_header={'backgroundColor': 'white', 'fontWeight': 'bold'})

predictions_div = html.Div(id='predictions', 
                           children=[html.H6(children='Products Recommended'),
                                     predictions],
                           style={'width': '25%'}, 
                           className="ten columns")

Overwriting layouts.py


##### Create the callbacks.py module

In [182]:
%%writefile callbacks.py

'''
This module contains the predict callback 

'''
from dash.dependencies import Input, Output

from app import *

@app.callback([Output('table', component_property='columns'), Output('table', component_property='data')],[Input(component_id='uid', component_property='value')])
def predict(uid):
    columns = []
    products_recommended = []
    if uid:
        model_path = locate_model(os.getcwd())
        predictions, _ = model_reader(model_path)
        uid_predictions = get_top_n_ui(get_top(predictions), uid)
        prediction_rank_lenght = len(uid_predictions)
        prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
        products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])
        columns=[{"name": i, "id": i} for i in products_recommended.columns]
        return columns, products_recommended.to_dict('records')
    else:
        return columns, products_recommended

Overwriting callbacks.py


##### Create the run.py module

In [183]:
%%writefile run.py

'''
This module runs the entire Dash Application

'''

import dash_table
import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output

from app import *
from layouts import c_id_div, predictions_div
import callbacks


app.layout= html.Div(
    
    id="score_gui", children=[
        html.H1('Recommendation System for Purchase Data'),
        html.H2('Scoring Interactive Web Service (Basic Concept)'),
        html.Br(), 
        html.Div(children=[c_id_div, predictions_div], className="row")
    ])

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Writing run.py


##### Write the requirements.txt

In [199]:
%%writefile requirements.txt

numpy
pandas
scikit-surprise
dash

Overwriting requirements.txt


##### Write Dockerfile

In [200]:
%%writefile Dockerfile

FROM python:3.8
    
WORKDIR /app

ADD . /app

RUN pip install -r ./requirements.txt

EXPOSE 8050

ENTRYPOINT [ "python" ]

CMD [ "run.py" ]

Overwriting Dockerfile


#### Run the Application with Python Docker client

In [201]:
image_name = "score-dash_app:1"

#Client instance
dockercli = docker.DockerClient()

#Check for image
if not dockercli.images.list(image_name):
    #if not build it
    dockercli.images.build(path='.', tag = image_name)
else:
    dockercli.images.remove(image_name, force = True)
    dockercli.images.build(path='.', tag = image_name)
try:
    app_container = dockercli.containers.run(image_name, name='scoring_dash_app_test', detach=True, ports={'8050/tcp': 8050})
    status = app_container.attrs
    print(status['State'])
    while (status['State']['Running'] == False):
        time.sleep(3)
        app_container.reload()
        status = app_container.attrs
        print(''.center(50, '-'))
        print(status['State'])
except RuntimeError as error:
    print(error)

{'Status': 'created', 'Running': False, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 0, 'ExitCode': 0, 'Error': '', 'StartedAt': '0001-01-01T00:00:00Z', 'FinishedAt': '0001-01-01T00:00:00Z'}
--------------------------------------------------
{'Status': 'running', 'Running': True, 'Paused': False, 'Restarting': False, 'OOMKilled': False, 'Dead': False, 'Pid': 494, 'ExitCode': 0, 'Error': '', 'StartedAt': '2020-06-05T16:18:08.700588489Z', 'FinishedAt': '0001-01-01T00:00:00Z'}


#### Kill the application

In [202]:
# stop and remove the container
app_container.stop()
app_container.remove()

#### To delete

In [169]:
%%writefile app.py
# -*- coding: utf-8 -*-

'''
This is the docstring for module/script.
'''

import os
import logging
from collections import defaultdict
import pandas as pd
from surprise import dump

import dash
import dash_table
import dash_core_components as component
import dash_html_components as html

from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions = True)
server = app.server

def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0

#Customerid Input
c_id_component = component.Input(id="uid", 
                                 type="text", 
                                 placeholder='100')

c_id_div = html.Div(id='customerid', children=[html.H6("Please enter your customer ID : "), 
                                               c_id_component,
                                               html.Br()], 
                   className="two columns")

#Prediction Output
predictions = dash_table.DataTable(id='table',
                                   css=[{'selector': 'table', 'rule': 'table-layout: fixed'}],
                                   style_as_list_view=True, 
                                   style_cell={'width': '30%', 'textAlign': 'left'},
                                   style_header={'backgroundColor': 'white', 'fontWeight': 'bold'})

predictions_div = html.Div(id='predictions', 
                           children=[html.H6(children='Products Recommended'),
                                     predictions],
                           style={'width': '25%'}, 
                           className="ten columns")
                           

app.layout= html.Div(
    
    id="score_gui", children=[
        html.H1('Recommendation System for Purchase Data'),
        html.H2('Scoring Interactive Web Service (Basic Concept)'),
        html.Br(), 
        html.Div(children=[c_id_div, predictions_div], className="row")
    ])

@app.callback([Output('table', component_property='columns'), Output('table', component_property='data')],[Input(component_id='uid', component_property='value')])
def predict(uid):
    columns = []
    products_recommended = []
    if uid:
        model_path = locate_model(os.getcwd())
        predictions, _ = model_reader(model_path)
        uid_predictions = get_top_n_ui(get_top(predictions), uid)
        prediction_rank_lenght = len(uid_predictions)
        prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
        products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])
        columns=[{"name": i, "id": i} for i in products_recommended.columns]
        return columns, products_recommended.to_dict('records')
    else:
        return columns, products_recommended

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting dash_app.py


In [15]:
%%writefile dash_app.py

# -*- coding: utf-8 -*-

'''
This is the docstring for module/script.
'''

import dash
import dash_core_components as component
import dash_html_components as html

external_stylesheets = ['https://codepen.io/ivannardini/pen/QWyLZJw.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

# Dash apps 1st element: layout

app.layout = html.Div(
    
    #Frame in the iphone cover
    id="iphoneCover", children=[
    
    #Insert a form
    html.Div(id="form_camp", children=[ 
    
    html.Form(children=[
        
        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', name='username', type='text')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_password', children=[
            html.Label('Password: '),
            component.Input(id='password', name='password', type='text')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_username', children=[
             html.Button('Login', type='submit')
        ])
    
    ], action='/predict', method='post')])
])

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting dash_app.py


In [61]:
%%writefile dash_app.py

'''
This is the docstring for module/script.
'''

import os
import logging
from collections import defaultdict
import pandas as pd
from surprise import dump

import dash
import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output


def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0


external_stylesheets = ['https://codepen.io/ivannardini/pen/QWyLZJw.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions = True)

# Page 1

app.layout = html.Div(
    
    #Frame in the iphone cover
    id="iphoneCover", children=[
    
    #Insert a form
    html.Div(id="form_camp", children=[ 
    
    html.Form(children=[
        
        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', name='uid', type='text', placeholder='Customer ID')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_password', children=[
            html.Label('Password: '),
            component.Input(id='password', type='password',  placeholder='Enter Password')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_username', children=[
             html.Button('Login', type='submit')
        ])
    
    ], action='/predictions', method='post')]), 
    
        html.Div(id="out1"),
        html.Div(id="out2")
    ])

@app.callback(
    [Output('out2', component_property='children'), 
     Output('out1', component_property='children')], 
    [Input(component_id='uid', component_property='value')]
)
def predict(uid):
#     logging.info('Scoring Application is starting to process the request')
    model_path = locate_model(os.getcwd())
    predictions, _ = model_reader(model_path)
    uid_predictions = get_top_n_ui(get_top(predictions), uid)

    prediction_rank_lenght = len(uid_predictions)
    prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
    products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])

    data = products_recommended.to_dict()
    prodname = data["Product_Rank"]
    prodid = data["Product_id"]
    
    return prodname, prodid

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)


Overwriting dash_app.py


In [136]:
%%writefile dash_app.py

'''
This is the docstring for module/script.
'''

import os
import logging
from collections import defaultdict
import pandas as pd
from surprise import dump

import dash
import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output, State


def locate_model(dest):
    
    """ 
    return path of binary model
    args:
       dest: folder for searching
    returns:
       model_path
    """

    for dirpath, dirnames, filenames in os.walk(dest):
        for filename in [f for f in filenames if f.endswith((".pkl", ".pickle"))]:
            model_path = os.path.join(dirpath, filename)
            return model_path
    return None

def model_reader(model_path):
    """ 
    return predictions and model class
    args:
       model_path: pickle file path
    returns:
       predictions and model
    """
    predictions, algo = dump.load(model_path)
    return predictions, algo

def get_top(predictions, n=10):
    
    '''
    Returns the the top-N recommendation from a set of predictions
    args:
       predictions: predictions generated in testing phase
       n: number of items to suggest (default=10)
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
        
    return top_n

def get_top_n_ui(top, uid):
    '''
    Returns the list of selected items
    args:
       top: user-prediction dictionaries
       uid: user id for filtering
    return:
       top_n dictionary: user-prediction dictionaries
    '''
    try:
        top_n_ui = [[iid for (iid, _) in user_ratings] for UID, user_ratings in top.items() if UID==uid][0]
        return top_n_ui
    except ValueError: 
        return 0


external_stylesheets = ['https://codepen.io/ivannardini/pen/QWyLZJw.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions = True)

#page 1 
app.layout = html.Div(
    
    #Frame in the iphone cover
    id="iphoneCover", children=[
        
#     component.Location(id='url', refresh=False),
    
    #Insert a form
    html.Div(id="form", children=[ 
    
    html.Form(children=[
        
        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', name='uid', type='text', placeholder='Customer ID')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_password', children=[
            html.Label('Password: '),
            component.Input(id='password', type='password',  placeholder='Enter Password')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_username', children=[
            html.Button('Login', type='submit', n_clicks=0)
        ])
    
    ])])
])


#Page 2 
score_layout = html.Div(id='form', children=[
        
        html.Form(children=[

        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', name='uid', type='text', placeholder='Customer ID')]), 

        html.Br(), 

        html.Div(id='form_camp_password', children=[
            html.Label('Suca: '),
            component.Input(id='password', type='password',  placeholder='Enter Password')]), 

        html.Br()])
])


@app.callback(Output('form', 'children'), [Input('login', 'n_clicks')]
def display_page(n_clicks):
    if n_clicks > 0:
        return score_layout
             )
        
# @app.callback([Output('out2', component_property='children'), Output('out1', component_property='children')],
#               [Input('login', 'n_clicks'), Input('url', 'pathname')])

# def predict(n_clicks, pathname):
#     if n_clicks > 0:
        
#         uid = pathname.split("=")[1].strip()
#         model_path = locate_model(os.getcwd())
#         predictions, _ = model_reader(model_path)
#         uid_predictions = get_top_n_ui(get_top(predictions), uid)

#         prediction_rank_lenght = len(uid_predictions)
#         prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
#         products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])

#         data = products_recommended.to_dict()
#         prodname = data["Product_Rank"]
#         prodid = data["Product_id"]
        
#         return str(prodname), str(prodid)

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting dash_app.py


In [119]:
%%writefile dash_app.py

import dash
import dash_html_components as html
import dash_core_components as dcc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div(dcc.Input(id='input-on-submit', type='text')),
    html.Button('Submit', id='submit-val', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a value and press submit')
])


@app.callback(
    dash.dependencies.Output('container-button-basic', 'children'),
    [dash.dependencies.Input('submit-val', 'n_clicks')],
    [dash.dependencies.State('input-on-submit', 'value')])
def update_output(n_clicks, value):
    return 'The input value was "{}" and the button has been clicked {} times'.format(
        value,
        n_clicks


if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting dash_app.py


In [152]:
app_folder = dash_folder + '/apps'
if not os.path.exists(app_folder):
    os.makedirs(app_folder)

with open(app_folder + '/__init__.py', 'w') as m:
    pass

##### Write app.py

In [153]:
%%writefile app.py

'''
Module to initiate Dash Server App
'''

import dash

external_stylesheets = ['https://codepen.io/ivannardini/pen/QWyLZJw.css']

app = dash.Dash(__name__, 
                external_stylesheets=external_stylesheets,
                 suppress_callback_exceptions = True)
server = app.server

Overwriting app.py


##### Write form_app.py

In [154]:
%%writefile ./apps/form_app1.py

'''
Module to render the form page
'''

import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output

from app import app

#Insert a Input form
form = html.Form(children=[
        
        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', type='text', placeholder='Customer ID')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_password', children=[
            html.Label('Password: '),
            component.Input(id='password', type='password',  placeholder='Enter Password')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_username', children=[
             html.Button('Login', type='submit')
        ])
    ])

# Dash apps 1st element: layout
layout = html.Div(
    
    #Frame in the iphone cover
    id="iphoneCover", children=[
        
        html.Div(id="form_camp", children=[form])
        
        html.Div(id="link")
        
])

# #Dash apps 2nd element: Callbacks
# @app.callback(Output('page-content', 'children'),
#               [Input('url', 'pathname')])
# def display_page(pathname):
#     return html.Div([
#         html.H3('You are on page {}'.format(pathname))
#     ])

Overwriting ./apps/form_app1.py


##### Write index.py

In [155]:
%%writefile index.py

'''
Module to execute the app
'''

import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output

from app import app
from apps import form_app1

form_app1.layout()

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting index.py


In [27]:
%%writefile dash_app.py

# -*- coding: utf-8 -*-

'''
This is the docstring for module/script.
'''

import os
import logging
from collections import defaultdict

import pandas as pd
from surprise import dump

import flask

import warnings
warnings.filterwarnings("ignore")

import dash
import dash_core_components as component
import dash_html_components as html
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/ivannardini/pen/QWyLZJw.css']

app = dash.Dash(__name__, 
                external_stylesheets=external_stylesheets,
                 suppress_callback_exceptions = True)

#Insert a Input form
form = html.Form(children=[
        
        html.Div(id='form_camp_customerid', children=[
            html.Label('Customer ID: '), 
            component.Input(id='uid', name='username', type='number')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_password', children=[
            html.Label('Password: '),
            component.Input(id='password', name='password', type='text')
        ]), 
        
        html.Br(), 
        
        html.Div(id='form_camp_username', children=[
             html.Button('Login', type='submit')
        ])
    ])

#Insert a Output form
product_table = html.Div([

    html.Table(id='product_table', children=[
    
        html.Tr([html.Td(id='productname'), html.Td(id='productid')])
    ])

# Dash apps 1st element: layout
app.layout = html.Div(
    
    #Record the url
    component.Location(id='url', refresh=False),
    
    #Frame in the iphone cover
    id="iphoneCover", children=[
        html.Div(id="form_camp", children=[
            
            form,
            product_table,
            
    ])
])
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page


# #Callback to retrive the uid value

# @app.callback(
#     [Output('productname', 'children'),
#      Output('productid', 'children')],     
#     [Input('uid', 'value')])
# def predict(uid):
    
#     logging.info('Scoring Application is starting to process the request')

#     model_path = locate_model(os.getcwd())
#     predictions, _ = model_reader(model_path)
#     uid_predictions = get_top_n_ui(get_top(predictions), uid)

#     prediction_rank_lenght = len(uid_predictions)
#     prediction_rank_labels = ["".join([" Product", str(i)]) for i in range(1,prediction_rank_lenght)]
#     products_recommended = pd.DataFrame(list(zip(prediction_rank_labels, uid_predictions)), columns=['Product_Rank', 'Product_id'])

#     data = products_recommended.to_dict()
#     prodname = data["Product_Rank"]["0"]
#     prodid = data["Product_id"]["0"]
    
#     return prodname, prodid

if __name__ == '__main__':
    app.run_server(host="0.0.0.0", debug=True)

Overwriting dash_app.py
